In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [2]:
base_model=MobileNetV2(weights='imagenet', include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

9412608/9406464 [==============================] - 3s 0us/step


In [3]:
model = Model(inputs=base_model.input, outputs=preds)

In [4]:
batch_size = 8

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 validation_split=0.3) #included in our dependencies

In [6]:
train_generator = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/dataset123',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)

Found 180 images belonging to 3 classes.


In [7]:
val_generator = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/dataset123',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  shuffle=True)

Found 75 images belonging to 3 classes.


In [8]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
callbacks = [ModelCheckpoint('model_chkpt/weights.{epoch:02d}_{val_loss:.4f}_{val_acc:.4f}.h5')]

In [12]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // batch_size,
                    callbacks=callbacks,
                    epochs=20,
                    )

Epoch 1/20
22/22 [==============================] - 24s 1s/step - loss: 0.1508 - accuracy: 0.9884 - val_loss: 0.6232 - val_accuracy: 0.7500


KeyError: 'Failed to format this callback filepath: "model_chkpt/weights.{epoch:02d}_{val_loss:.4f}_{val_acc:.4f}.h5". Reason: \'val_acc\''